In [1]:
import pyspiel
from open_spiel.python.algorithms import cfr
from open_spiel.python.algorithms import expected_game_score

In [16]:

game = pyspiel.load_game("universal_poker")

games_list = pyspiel.registered_games()

print("Registered games:")
print(games_list)


Registered games:
[<GameType 'backgammon'>, <GameType 'blotto'>, <GameType 'breakthrough'>, <GameType 'bridge_uncontested_bidding'>, <GameType 'catch'>, <GameType 'chess'>, <GameType 'coin_game'>, <GameType 'connect_four'>, <GameType 'coop_box_pushing'>, <GameType 'first_sealed_auction'>, <GameType 'go'>, <GameType 'goofspiel'>, <GameType 'havannah'>, <GameType 'hex'>, <GameType 'kuhn_poker'>, <GameType 'laser_tag'>, <GameType 'leduc_poker'>, <GameType 'liars_dice'>, <GameType 'markov_soccer'>, <GameType 'matching_pennies_3p'>, <GameType 'matrix_cd'>, <GameType 'matrix_coordination'>, <GameType 'matrix_mp'>, <GameType 'matrix_pd'>, <GameType 'matrix_rps'>, <GameType 'matrix_sh'>, <GameType 'matrix_shapleys_game'>, <GameType 'misere'>, <GameType 'negotiation'>, <GameType 'oshi_zumo'>, <GameType 'oware'>, <GameType 'pentago'>, <GameType 'phantom_ttt'>, <GameType 'pig'>, <GameType 'quoridor'>, <GameType 'tic_tac_toe'>, <GameType 'tiny_bridge_2p'>, <GameType 'tiny_bridge_4p'>, <GameType 't

In [22]:
"""Test that Python and C++ bots can be called by a C++ algorithm."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl.testing import absltest
import numpy as np

from open_spiel.python.bots import uniform_random
import pyspiel

game = pyspiel.load_game("leduc_poker")
bots = [
    pyspiel.make_uniform_random_bot(game, 0, 1234),
    uniform_random.UniformRandomBot(game, 1, np.random.RandomState(4321)),
]
results = np.array([
    pyspiel.evaluate_bots(game.new_initial_state(), bots, iteration)
    for iteration in range(10000)
])
leduc_average_results = np.mean(results, axis=0)
print(leduc_average_results)

game = pyspiel.load_game("universal_poker")
bots = [
    pyspiel.make_uniform_random_bot(game, 0, 1234),
    uniform_random.UniformRandomBot(game, 1, np.random.RandomState(4321)),
]
results = np.array([
    pyspiel.evaluate_bots(game.new_initial_state(), bots, iteration)
    for iteration in range(10000)
])
universal_poker_average_results = np.mean(results, axis=0)
print(universal_poker_average_results)
    
np.testing.assert_allclose(universal_poker_average_results, leduc_average_results, atol=0.1)

[-0.0998  0.0998]
[-0.0998  0.0998]


In [34]:
"""Python spiel example."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pickle
from absl.testing import absltest
from absl.testing import parameterized
import numpy as np

import pyspiel

# Put a bound on length of game so test does not timeout.
MAX_ACTIONS_PER_GAME = 1000

# All games registered in the main spiel library.
SPIEL_GAMES_LIST = pyspiel.registered_games()


# Check for mandatory parameters.
def _has_mandatory_params(game):
    return any(p.is_mandatory() for p in game.parameter_specification.values())


# All games without mandatory parameters.
SPIEL_LOADABLE_GAMES_LIST = [
    g for g in SPIEL_GAMES_LIST if not _has_mandatory_params(g)
]
print(SPIEL_LOADABLE_GAMES_LIST)
# TODO(b/141950198): Stop hard-coding the number of loadable games.
assert len(SPIEL_LOADABLE_GAMES_LIST) >= 38, len(SPIEL_LOADABLE_GAMES_LIST)

# All simultaneous games.
SPIEL_SIMULTANEOUS_GAMES_LIST = [
    g for g in SPIEL_LOADABLE_GAMES_LIST
    if g.dynamics == pyspiel.GameType.Dynamics.SIMULTANEOUS
]
assert len(SPIEL_SIMULTANEOUS_GAMES_LIST) >= 14, len(
    SPIEL_SIMULTANEOUS_GAMES_LIST)

# All multiplayer games. This is a list of (game, num_players) pairs to test.
SPIEL_MULTIPLAYER_GAMES_LIST = [
    # pylint: disable=g-complex-comprehension
    (g, p)
    for g in SPIEL_LOADABLE_GAMES_LIST
    for p in range(max(g.min_num_players, 2), 1 + min(g.max_num_players, 6))
    if g.max_num_players > 2 and g.max_num_players > g.min_num_players
    and g.short_name != "tiny_hanabi"  # default payoff only works for 2p
]
assert len(SPIEL_MULTIPLAYER_GAMES_LIST) >= 35, len(
    SPIEL_MULTIPLAYER_GAMES_LIST)


[<GameType 'backgammon'>, <GameType 'blotto'>, <GameType 'breakthrough'>, <GameType 'bridge_uncontested_bidding'>, <GameType 'catch'>, <GameType 'chess'>, <GameType 'coin_game'>, <GameType 'connect_four'>, <GameType 'coop_box_pushing'>, <GameType 'first_sealed_auction'>, <GameType 'go'>, <GameType 'goofspiel'>, <GameType 'havannah'>, <GameType 'hex'>, <GameType 'kuhn_poker'>, <GameType 'laser_tag'>, <GameType 'leduc_poker'>, <GameType 'liars_dice'>, <GameType 'markov_soccer'>, <GameType 'matching_pennies_3p'>, <GameType 'matrix_cd'>, <GameType 'matrix_coordination'>, <GameType 'matrix_mp'>, <GameType 'matrix_pd'>, <GameType 'matrix_rps'>, <GameType 'matrix_sh'>, <GameType 'matrix_shapleys_game'>, <GameType 'negotiation'>, <GameType 'oshi_zumo'>, <GameType 'oware'>, <GameType 'pentago'>, <GameType 'phantom_ttt'>, <GameType 'pig'>, <GameType 'quoridor'>, <GameType 'tic_tac_toe'>, <GameType 'tiny_bridge_2p'>, <GameType 'tiny_bridge_4p'>, <GameType 'tiny_hanabi'>, <GameType 'universal_poke

In [29]:
class GamesSimTest(parameterized.TestCase):

  def apply_action(self, state, action):
    if state.is_simultaneous_node():
      state.apply_actions(action)
    else:
      state.apply_action(action)

  def apply_action_test_clone(self, state, action):
    """Apply the action and test the clone method if it's implemented."""
    try:
      state_clone = state.clone()
      self.assertEqual(str(state), str(state_clone))
      self.assertEqual(state.history(), state_clone.history())
      self.apply_action(state, action)
      self.apply_action(state_clone, action)
      self.assertEqual(str(state), str(state_clone))
      self.assertEqual(state.history(), state_clone.history())
    except Exception:  # pylint: disable=broad-except
      self.apply_action(state, action)

  def serialize_deserialize(self, game, state):
    # OpenSpiel native serialization
    ser_str = pyspiel.serialize_game_and_state(game, state)
    new_game, new_state = pyspiel.deserialize_game_and_state(ser_str)
    self.assertEqual(str(game), str(new_game))
    self.assertEqual(str(state), str(new_state))
    # Pickle serialization + deserialization (of the state).
    pickled_state = pickle.dumps(state)
    unpickled_state = pickle.loads(pickled_state)
    self.assertEqual(str(state), str(unpickled_state))

  def sim_game(self, game):
    min_utility = game.min_utility()
    max_utility = game.max_utility()
    self.assertLess(min_utility, max_utility)

    # Pickle serialization + deserialization (of the game).
    pickled_game = pickle.dumps(game)
    unpickled_game = pickle.loads(pickled_game)
    self.assertEqual(str(game), str(unpickled_game))

    # Get a new state
    state = game.new_initial_state()
    total_actions = 0

    next_serialize_check = 1

    while not state.is_terminal() and total_actions <= MAX_ACTIONS_PER_GAME:
      total_actions += 1

      # Serialize/Deserialize is costly. Only do it every power of 2 actions.
      if total_actions >= next_serialize_check:
        self.serialize_deserialize(game, state)
        next_serialize_check *= 2

      # The state can be three different types: chance node,
      # simultaneous node, or decision node
      if state.is_chance_node():
        # Chance node: sample an outcome
        outcomes = state.chance_outcomes()
        self.assertNotEmpty(outcomes)
        action_list, prob_list = zip(*outcomes)
        action = np.random.choice(action_list, p=prob_list)
        state.apply_action(action)
      elif state.is_simultaneous_node():
        # Simultaneous node: sample actions for all players
        chosen_actions = [
            np.random.choice(state.legal_actions(pid))
            for pid in range(game.num_players())
        ]
        # Apply the joint action and test cloning states.
        self.apply_action_test_clone(state, chosen_actions)
      else:
        # Decision node: sample action for the single current player
        action = np.random.choice(state.legal_actions(state.current_player()))
        # Apply action and test state cloning.
        self.apply_action_test_clone(state, action)

    # Max sure at least one action was made.
    self.assertGreater(total_actions, 0,
                       "No actions taken in sim of " + str(game))

    # Either the game is now done, or the maximum actions has been taken.
    if state.is_terminal():
      # Check there are no legal actions.
      self.assertEmpty(state.legal_actions())
      for player in range(game.num_players()):
        self.assertEmpty(state.legal_actions(player))
      # Print utilities for each player.
      utilities = state.returns()
      # Check that each one is in range
      for utility in utilities:
        self.assertGreaterEqual(utility, game.min_utility())
        self.assertLessEqual(utility, game.max_utility())
      print("Sim of game {} terminated with {} total actions. Utilities: {}"
            .format(game, total_actions, utilities))
    else:
      print(
          "Sim of game {} terminated after maximum number of actions {}".format(
              game, MAX_ACTIONS_PER_GAME))

  @parameterized.parameters(*SPIEL_LOADABLE_GAMES_LIST)
  def test_game_sim(self, game_info):
    game = pyspiel.load_game(game_info.short_name)
    self.assertLessEqual(game_info.min_num_players, game.num_players())
    self.assertLessEqual(game.num_players(), game_info.max_num_players)
    self.sim_game(game)

  @parameterized.parameters(*SPIEL_SIMULTANEOUS_GAMES_LIST)
  def test_simultaneous_game_as_turn_based(self, game_info):
    converted_game = pyspiel.load_game_as_turn_based(game_info.short_name)
    self.sim_game(converted_game)

  @parameterized.parameters(*SPIEL_MULTIPLAYER_GAMES_LIST)
  def test_multiplayer_game(self, game_info, num_players):
    game = pyspiel.load_game(game_info.short_name,
                             {"players": pyspiel.GameParameter(num_players)})
    self.sim_game(game)

  def test_breakthrough(self):
    # make a smaller (6x6) board
    game = pyspiel.load_game("breakthrough(rows=6,columns=6)")
    self.sim_game(game)

  def test_pig(self):
    # make a smaller lower win score
    game = pyspiel.load_game("pig(players=2,winscore=15)")
    self.sim_game(game)

In [32]:
test_game_sim(self, game_info)

NameError: name 'self' is not defined

In [35]:
# Copyright 2019 DeepMind Technologies Ltd. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Python spiel example."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pickle
from absl.testing import absltest
from absl.testing import parameterized
import numpy as np

import pyspiel

# Put a bound on length of game so test does not timeout.
MAX_ACTIONS_PER_GAME = 1000

# All games registered in the main spiel library.
SPIEL_GAMES_LIST = pyspiel.registered_games()


# Check for mandatory parameters.
def _has_mandatory_params(game):
  return any(p.is_mandatory() for p in game.parameter_specification.values())


# All games without mandatory parameters.
SPIEL_LOADABLE_GAMES_LIST = [
    g for g in SPIEL_GAMES_LIST if not _has_mandatory_params(g)
]
# TODO(b/141950198): Stop hard-coding the number of loadable games.
assert len(SPIEL_LOADABLE_GAMES_LIST) >= 38, len(SPIEL_LOADABLE_GAMES_LIST)

# All simultaneous games.
SPIEL_SIMULTANEOUS_GAMES_LIST = [
    g for g in SPIEL_LOADABLE_GAMES_LIST
    if g.dynamics == pyspiel.GameType.Dynamics.SIMULTANEOUS
]
assert len(SPIEL_SIMULTANEOUS_GAMES_LIST) >= 14, len(
    SPIEL_SIMULTANEOUS_GAMES_LIST)

# All multiplayer games. This is a list of (game, num_players) pairs to test.
SPIEL_MULTIPLAYER_GAMES_LIST = [
    # pylint: disable=g-complex-comprehension
    (g, p)
    for g in SPIEL_LOADABLE_GAMES_LIST
    for p in range(max(g.min_num_players, 2), 1 + min(g.max_num_players, 6))
    if g.max_num_players > 2 and g.max_num_players > g.min_num_players
    and g.short_name != "tiny_hanabi"  # default payoff only works for 2p
]
assert len(SPIEL_MULTIPLAYER_GAMES_LIST) >= 35, len(
    SPIEL_MULTIPLAYER_GAMES_LIST)


class GamesSimTest(parameterized.TestCase):

  def apply_action(self, state, action):
    if state.is_simultaneous_node():
      state.apply_actions(action)
    else:
      state.apply_action(action)

  def apply_action_test_clone(self, state, action):
    """Apply the action and test the clone method if it's implemented."""
    try:
      state_clone = state.clone()
      self.assertEqual(str(state), str(state_clone))
      self.assertEqual(state.history(), state_clone.history())
      self.apply_action(state, action)
      self.apply_action(state_clone, action)
      self.assertEqual(str(state), str(state_clone))
      self.assertEqual(state.history(), state_clone.history())
    except Exception:  # pylint: disable=broad-except
      self.apply_action(state, action)

  def serialize_deserialize(self, game, state):
    # OpenSpiel native serialization
    ser_str = pyspiel.serialize_game_and_state(game, state)
    new_game, new_state = pyspiel.deserialize_game_and_state(ser_str)
    self.assertEqual(str(game), str(new_game))
    self.assertEqual(str(state), str(new_state))
    # Pickle serialization + deserialization (of the state).
    pickled_state = pickle.dumps(state)
    unpickled_state = pickle.loads(pickled_state)
    self.assertEqual(str(state), str(unpickled_state))

  def sim_game(self, game):
    min_utility = game.min_utility()
    max_utility = game.max_utility()
    self.assertLess(min_utility, max_utility)

    # Pickle serialization + deserialization (of the game).
    pickled_game = pickle.dumps(game)
    unpickled_game = pickle.loads(pickled_game)
    self.assertEqual(str(game), str(unpickled_game))

    # Get a new state
    state = game.new_initial_state()
    total_actions = 0

    next_serialize_check = 1

    while not state.is_terminal() and total_actions <= MAX_ACTIONS_PER_GAME:
      total_actions += 1

      # Serialize/Deserialize is costly. Only do it every power of 2 actions.
      if total_actions >= next_serialize_check:
        self.serialize_deserialize(game, state)
        next_serialize_check *= 2

      # The state can be three different types: chance node,
      # simultaneous node, or decision node
      if state.is_chance_node():
        # Chance node: sample an outcome
        outcomes = state.chance_outcomes()
        self.assertNotEmpty(outcomes)
        action_list, prob_list = zip(*outcomes)
        action = np.random.choice(action_list, p=prob_list)
        state.apply_action(action)
      elif state.is_simultaneous_node():
        # Simultaneous node: sample actions for all players
        chosen_actions = [
            np.random.choice(state.legal_actions(pid))
            for pid in range(game.num_players())
        ]
        # Apply the joint action and test cloning states.
        self.apply_action_test_clone(state, chosen_actions)
      else:
        # Decision node: sample action for the single current player
        action = np.random.choice(state.legal_actions(state.current_player()))
        # Apply action and test state cloning.
        self.apply_action_test_clone(state, action)

    # Max sure at least one action was made.
    self.assertGreater(total_actions, 0,
                       "No actions taken in sim of " + str(game))

    # Either the game is now done, or the maximum actions has been taken.
    if state.is_terminal():
      # Check there are no legal actions.
      self.assertEmpty(state.legal_actions())
      for player in range(game.num_players()):
        self.assertEmpty(state.legal_actions(player))
      # Print utilities for each player.
      utilities = state.returns()
      # Check that each one is in range
      for utility in utilities:
        self.assertGreaterEqual(utility, game.min_utility())
        self.assertLessEqual(utility, game.max_utility())
      print("Sim of game {} terminated with {} total actions. Utilities: {}"
            .format(game, total_actions, utilities))
    else:
      print(
          "Sim of game {} terminated after maximum number of actions {}".format(
              game, MAX_ACTIONS_PER_GAME))

  @parameterized.parameters(*SPIEL_LOADABLE_GAMES_LIST)
  def test_game_sim(self, game_info):
    game = pyspiel.load_game(game_info.short_name)
    self.assertLessEqual(game_info.min_num_players, game.num_players())
    self.assertLessEqual(game.num_players(), game_info.max_num_players)
    self.sim_game(game)

  @parameterized.parameters(*SPIEL_SIMULTANEOUS_GAMES_LIST)
  def test_simultaneous_game_as_turn_based(self, game_info):
    converted_game = pyspiel.load_game_as_turn_based(game_info.short_name)
    self.sim_game(converted_game)

  @parameterized.parameters(*SPIEL_MULTIPLAYER_GAMES_LIST)
  def test_multiplayer_game(self, game_info, num_players):
    game = pyspiel.load_game(game_info.short_name,
                             {"players": pyspiel.GameParameter(num_players)})
    self.sim_game(game)

  def test_breakthrough(self):
    # make a smaller (6x6) board
    game = pyspiel.load_game("breakthrough(rows=6,columns=6)")
    self.sim_game(game)

  def test_pig(self):
    # make a smaller lower win score
    game = pyspiel.load_game("pig(players=2,winscore=15)")
    self.sim_game(game)


if __name__ == "__main__":
  absltest.main()

Running tests under Python 3.7.3: /home/eugen/anaconda3/bin/python
FATAL Flags parsing error: Unknown command line flag 'f'
Pass --helpshort or --helpfull to see help on flags.


SystemExit: 1

In [53]:
!python ~/Documents/open_spiel/open_spiel/python/tests/games_sim_test.py  

Running tests under Python 3.7.3: /home/eugen/anaconda3/bin/python
[ RUN      ] GamesSimTest.test_breakthrough
Sim of game breakthrough(columns=6,rows=6) terminated with 46 total actions. Utilities: [-1.0, 1.0]
[       OK ] GamesSimTest.test_breakthrough
[ RUN      ] GamesSimTest.test_game_sim(<GameType 'backgammon'>)
Sim of game backgammon() terminated with 228 total actions. Utilities: [-1.0, 1.0]
[       OK ] GamesSimTest.test_game_sim(<GameType 'backgammon'>)
[ RUN      ] GamesSimTest.test_game_sim(<GameType 'blotto'>)
Sim of game blotto() terminated with 1 total actions. Utilities: [-1.0, 1.0]
[       OK ] GamesSimTest.test_game_sim(<GameType 'blotto'>)
[ RUN      ] GamesSimTest.test_game_sim(<GameType 'go'>)
Sim of game go() terminated with 510 total actions. Utilities: [-1.0, 1.0]
[       OK ] GamesSimTest.test_game_sim(<GameType 'go'>)
[ RUN      ] GamesSimTest.test_game_sim(<GameType 'goofspiel'>)
Sim of game goofspiel() terminated with 24 total actions. Utilities: [1.0, -1.0]

Sim of game coop_box_pushing() terminated with 400 total actions. Utilities: [-70.0, -70.0]
[       OK ] GamesSimTest.test_game_sim(<GameType 'coop_box_pushing'>)
[ RUN      ] GamesSimTest.test_game_sim(<GameType 'first_sealed_auction'>)
Sim of game first_sealed_auction() terminated with 5 total actions. Utilities: [3.0, 0.0]
[       OK ] GamesSimTest.test_game_sim(<GameType 'first_sealed_auction'>)
[ RUN      ] GamesSimTest.test_multiplayer_game(<GameType 'blotto'>, 2)
Sim of game blotto(players=2) terminated with 1 total actions. Utilities: [-1.0, 1.0]
[       OK ] GamesSimTest.test_multiplayer_game(<GameType 'blotto'>, 2)
[ RUN      ] GamesSimTest.test_multiplayer_game(<GameType 'blotto'>, 3)
Sim of game blotto(players=3) terminated with 1 total actions. Utilities: [-1.0, 0.5, 0.5]
[       OK ] GamesSimTest.test_multiplayer_game(<GameType 'blotto'>, 3)
[ RUN      ] GamesSimTest.test_multiplayer_game(<GameType 'first_sealed_auction'>, 2)
Sim of game first_sealed_auction(players=2) te

Sim of game universal_poker(players=6) terminated with 18 total actions. Utilities: [-3.0, -1.0, 15.0, -7.0, -1.0, -3.0]
[       OK ] GamesSimTest.test_multiplayer_game(<GameType 'universal_poker'>, 6)
[ RUN      ] GamesSimTest.test_multiplayer_game(<GameType 'blotto'>, 6)
Sim of game blotto(players=6) terminated with 1 total actions. Utilities: [-0.3333333333333333, -0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, -0.3333333333333333]
[       OK ] GamesSimTest.test_multiplayer_game(<GameType 'blotto'>, 6)
[ RUN      ] GamesSimTest.test_multiplayer_game(<GameType 'coin_game'>, 2)
Sim of game coin_game(players=2) terminated with 36 total actions. Utilities: [16.0, 16.0]
[       OK ] GamesSimTest.test_multiplayer_game(<GameType 'coin_game'>, 2)
[ RUN      ] GamesSimTest.test_multiplayer_game(<GameType 'coin_game'>, 3)
Sim of game coin_game(players=3) terminated with 42 total actions. Utilities: [1.0, 3.0, 1.0]
[       OK ] GamesSimTest.test_multiplayer_gam

In [54]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import app
from absl import flags

from open_spiel.python.algorithms import cfr
from open_spiel.python.algorithms import exploitability
import pyspiel

FLAGS = flags.FLAGS

flags.DEFINE_integer("iterations", 100, "Number of iterations")
flags.DEFINE_string("game", "kuhn_poker", "Name of the game")
flags.DEFINE_integer("players", 2, "Number of players")
flags.DEFINE_integer("print_freq", 10, "How often to print the exploitability")


def main(_):
  game = pyspiel.load_game(FLAGS.game,
                           {"players": pyspiel.GameParameter(FLAGS.players)})
  cfr_solver = cfr.CFRSolver(game)

  for i in range(FLAGS.iterations):
    cfr_solver.evaluate_and_update_policy()
    if i % FLAGS.print_freq == 0:
      conv = exploitability.exploitability(game, cfr_solver.average_policy())
      print("Iteration {} exploitability {}".format(i, conv))


if __name__ == "__main__":
  app.run(main)

FATAL Flags parsing error: Unknown command line flag 'f'
Pass --helpshort or --helpfull to see help on flags.


SystemExit: 1

In [66]:

# flags.DEFINE_integer("iterations", 100, "Number of iterations")
# flags.DEFINE_string("game", "kuhn_poker", "Name of the game")
# flags.DEFINE_integer("players", 2, "Number of players")
# flags.DEFINE_integer("print_freq", 10, "How often to print the exploitability")

# Compare exloitability for two games

game_1 = "kuhn_poker"
game_2 = "universal_poker"
players = 2
iterations = 100
print_freq = 10

def compare_exploitability(game_1 = game_1, game_2 = game_2):
    game_1 = pyspiel.load_game(game_1,
                           {"players": pyspiel.GameParameter(players)})
    cfr_solver_1 = cfr.CFRSolver(game_1)
    game_2 = pyspiel.load_game(game_2,
                           {"players": pyspiel.GameParameter(players)})
    cfr_solver_2 = cfr.CFRSolver(game_2)
    
    for i in range(iterations):
        cfr_solver_1.evaluate_and_update_policy()
        cfr_solver_2.evaluate_and_update_policy()
        if i % print_freq == 0:
            conv_1 = exploitability.exploitability(game_1, cfr_solver_1.average_policy())
            conv_2 = exploitability.exploitability(game_2, cfr_solver_2.average_policy())


            print("Iteration {} exploitability of the {}: {} {}: {}".format(i, game_1, game_2, conv_1, conv_2))

    print("Final exploitability is {}: {} {}: {}".format(game_1, game_2, conv_1, conv_2))
    
compare_exploitability(game_1, game_2)

Iteration 0 exploitability of the kuhn_poker(players=2): universal_poker(players=2) 0.45833333333333326: 2.373611111111111
Iteration 10 exploitability of the kuhn_poker(players=2): universal_poker(players=2) 0.06046924690611866: 0.82051887735123
Iteration 20 exploitability of the kuhn_poker(players=2): universal_poker(players=2) 0.039914275345009825: 0.4318900723263599
Iteration 30 exploitability of the kuhn_poker(players=2): universal_poker(players=2) 0.024167348753902612: 0.2974616802751786
Iteration 40 exploitability of the kuhn_poker(players=2): universal_poker(players=2) 0.020517348345035824: 0.22759619868737918
Iteration 50 exploitability of the kuhn_poker(players=2): universal_poker(players=2) 0.014479024570810684: 0.1816384526206794
Iteration 60 exploitability of the kuhn_poker(players=2): universal_poker(players=2) 0.014003542854660017: 0.15598665979550697
Iteration 70 exploitability of the kuhn_poker(players=2): universal_poker(players=2) 0.011778275229671564: 0.1330410351397